In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
# 1. 설정 및 재료 준비
# ---------------------------------------------------------
num_rows = 200  # 목표 데이터 개수
start_date = datetime(2026, 1, 1) # 2026년 1월

# 랜덤으로 뽑을 카테고리에서 '주거/통신' 제거 (따로 만들 예정)
category_map_random = {
    '식비': ['스타벅스', '맥도날드', '김밥천국', '배달의민족', '이마트24', 'GS25', '투썸플레이스', '쿠팡이츠', '구내식당', '삼겹살집'],
    '교통비': ['지하철', '택시', '버스', 'SK주유소', '카카오T', '코레일', 'GS칼텍스'],
    '쇼핑': ['쿠팡', '다이소', '무신사', '올리브영', '네이버쇼핑', '유니클로', '무인양품', 'ZARA'],
    '구독': ['넷플릭스', '유튜브프리미엄', '멜론', '쿠팡와우', 'ChatGPT'],
    '의료/건강': ['약국', '내과', '헬스장할부', '치과', '정형외과'],
    '문화/여가': ['CGV', '교보문고', '롯데월드', 'PC방', '볼링장', '야구장'],
    '교육': ['토익접수', '교재구입', '인프런', '영어 강의'],
    '기타': ['모임회비', '경조사비', '더치페이']
}

# 주거/통신 항목은 별도 리스트로 정의 (1회씩만 등장)
housing_items = ['월세', '관리비', 'SKT통신비', '도시가스', '인터넷요금']

payment_methods = ['신용카드', '체크카드', '계좌이체', '현금']

# 카테고리 가중치 ('주거/통신' 빠진 만큼 비율 재조정)
categories = list(category_map_random.keys())
# 순서: 식비, 교통, 쇼핑, 구독, 의료, 문화, 교육, 기타
weights = [0.35, 0.21, 0.16, 0.05, 0.05, 0.13, 0.03, 0.02] 



In [3]:
# 2. 데이터 생성 함수
# ---------------------------------------------------------
def modified_date(base_date):
    """날짜 포맷 변환 및 결측치 생성"""
    r = random.random()
    if r < 0.001: return np.nan
    rand_day = random.randint(0, 31)
    target_date = base_date + timedelta(days=rand_day)
    
    if r < 0.8: return target_date.strftime("%Y-%m-%d")
    elif r < 0.9: return target_date.strftime("%m.%d")
    else: return target_date.strftime("%Y/%m/%d")

def modified_amount(category, fixed_value=None):
    """금액 포맷 변환"""
    r = random.random()
    if r < 0.001: return np.nan
    
    if fixed_value is not None:
        base_amount = fixed_value
    else:
        # 랜덤 카테고리별 금액 범위
        if category == '교육':
            base_amount = random.randint(5, 30) * 10000
        elif category == '쇼핑':
            base_amount = random.randint(1, 20) * 10000
        elif category in ['식비', '교통비']:
            base_amount = random.randint(4, 50) * 1000
        elif category in '기타':
            base_amount = random.randint(10, 50) * 1000
        else:
            base_amount = random.randint(1, 10) * 1000
        
    if r < 0.6: return base_amount
    elif r < 0.9: return f"{base_amount:,}"
    else: return str(base_amount)

In [4]:

# 3. 데이터 조립
# ---------------------------------------------------------
data = []

#  '주거/통신' 데이터 생성 (항목별 1회씩만 생성하도록 설정)
for item in housing_items:
    # 1. 금액 설정 (항목별 현실적인 금액 지정)
    if item == '월세':
        price = 700000
        fix_date = datetime(2026, 1, 25) # 월세는 25일
    elif item == '관리비':
        price = random.randint(10, 20) * 10000 # 10~20만
        fix_date = None
    elif item == 'SKT통신비':
        price = random.randint(5, 10) * 10000 # 5~10만
        fix_date = None
    elif item == '인터넷요금':
        price = random.randint(2, 4) * 10000 # 2~4만
        fix_date = None
    else: # 도시가스 등
        price = random.randint(1, 5) * 10000
        fix_date = None
    
    # 2. 날짜 생성 (월세만 고정, 나머지는 랜덤)
    if fix_date:
        date_str = fix_date.strftime("%Y-%m-%d")
    else:
        date_str = modified_date(start_date)

    # 3. 포맷 적용
    amount_val = modified_amount('주거/통신', fixed_value=price)
    
    # 4. 추가 (고정비 True, 메모에 '고정지출' 표시)
    data.append([date_str, amount_val, '주거/통신', item, '계좌이체', True, '고정지출'])


# 나머지 데이터 랜덤 생성 (목표 개수 - 주거/통신 개수 만큼)
remaining_rows = num_rows - len(housing_items)

for _ in range(remaining_rows):
    # 1. 카테고리 선정 (주거/통신 제외됨)
    cat = random.choices(categories, weights=weights, k=1)[0]
    desc = random.choice(category_map_random[cat])
    
    # 2. 결측치 함정
    final_cat = np.nan if random.random() < 0.01 else cat

    # 3. 날짜/금액
    date_val = modified_date(start_date)
    amount_val = modified_amount(cat)
    pay_method = random.choice(payment_methods)
    
    if cat == '구독':
        is_fixed = random.choice([True, False])
    else:
        is_fixed = False
        
    memo = "확인필요" if random.random() > 0.95 else np.nan

    data.append([date_val, amount_val, final_cat, desc, pay_method, is_fixed, memo])

#섞기
random.shuffle(data)

# DataFrame 생성
df = pd.DataFrame(data, columns=['date', 'amount', 'category', 'description', 'payment_method', 'is_fixed', 'memo'])


# 4. 결과 저장
# ---------------------------------------------------------
#기타 항목 확인
print(df[df['category'] == '기타'][['date', 'category', 'description', 'amount']].head())

# CSV 저장
filename = "expense_data.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"\n파일 생성 완료")

In [5]:
# 4. 결과 저장
# ---------------------------------------------------------

print(df[df['category'] == '기타'][['date', 'category', 'description', 'amount']].head())

# CSV 저장
filename = "expense_data.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"\n파일 생성 완료")

           date category description  amount
14   2026-01-12       기타        경조사비   40000
51   2026/01/21       기타        모임회비   12000
84   2026-01-20       기타        모임회비  25,000
97   2026-01-17       기타        더치페이   49000
185  2026-01-01       기타        모임회비  49,000

파일 생성 완료
